Problem 4
========

In [35]:
import torch
import torch.nn as nn
from torchinfo import summary
from torchviz import make_dot
import torch
from torch.utils.data import Dataset,DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np


In [49]:
class STMConvLayer(nn.Module):
    def __init__(self):
        super(STMConvLayer, self).__init__()
        self.conv = nn.ModuleList()
        for _ in range(32):
            self.conv.append(
                nn.Sequential(
                    nn.Conv2d(256, 4, 1, dtype=torch.float),  # Specify dtype=torch.float
                    nn.ReLU(),
                    nn.Conv2d(4, 4, 3, padding=1, dtype=torch.float),  # Specify dtype=torch.float
                    nn.ReLU(),
                    nn.Conv2d(4, 256, 1, dtype=torch.float)  # Specify dtype=torch.float
                )
            )
    def forward(self,x): # x : bathced data
        output = torch.zeros(x.shape, dtype=torch.float)
        for path in self.conv:
            output += path(x.float())
        return output

In [52]:
d = 256
n_train = 100
dummy_data = np.random.randn(10, 256, 3, 3)  # 10 samples, 256 channels, 3x3 spatial dimensions

# Convert dummy data to PyTorch tensor
dummy_tensor = torch.tensor(dummy_data, dtype=torch.float)

# Create DataLoader with the dummy dataset
dummy_loader = DataLoader(dummy_tensor, batch_size=1)  # Batch size 1 as we're already providing batched data

batch = next(iter(dummy_loader))
model = STMConvLayer()
yhat = model(batch)
make_dot(yhat, params=dict(list(model.named_parameters()))).render("STMConvLayer", format="png")

'STMConvLayer.png'

In [59]:
print(model)

STMConvLayer(
  (conv): ModuleList(
    (0-31): 32 x Sequential(
      (0): Conv2d(256, 4, kernel_size=(1, 1), stride=(1, 1))
      (1): ReLU()
      (2): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU()
      (4): Conv2d(4, 256, kernel_size=(1, 1), stride=(1, 1))
    )
  )
)


Problem 5
=====

In [104]:
import matplotlib.pyplot as plt 
import numpy as np 


"""
Step 1 : Generate Toy data
"""

d = 35
n_train, n_val, n_test = 300, 60, 30
np.random.seed(0)
beta = np.random.randn(d)
beta_true = beta / np.linalg.norm(beta)
# Generate and fix training data
X_train = np.array([np.random.multivariate_normal(np.zeros(d), np.identity(d)) for _ in range(n_train)])
Y_train = X_train @ beta_true + np.random.normal(loc = 0.0, scale = 0.5, size = n_train)
# Generate and fix validation data (for tuning lambda). 
X_val = np.array([np.random.multivariate_normal(np.zeros(d), np.identity(d)) for _ in range(n_val)])
Y_val = X_val @ beta_true 
# Generate and fix test data
X_test = np.array([np.random.multivariate_normal(np.zeros(d), np.identity(d)) for _ in range(n_test)])
Y_test = X_test @ beta_true 


"""
Step 2 : Solve the problem
"""    
    
lambda_list = [2 ** i for i in range(-6, 6)]
num_params = np.arange(1,1501,10)

errors_opt_lambda = []
errors_fixed_lambda = []
for p in num_params : 
    W = np.random.normal(0, 1/p, (p, d))
    Relu = np.vectorize(lambda z: np.fmax(0,z))
    phi = Relu(X_train @ W.T)
    phi_val = Relu(X_val @ W.T)
    phi_test = Relu(X_test @ W.T)
    lambda_opt = min(lambda_list, key = lambda lamb : np.linalg.norm(Y_val - phi_val @ np.linalg.pinv(phi_val.T @ phi_val + lamb * np.eye(p)) @ phi_val.T @ Y_val))
    
    theta_fixed = np.linalg.pinv(phi.T @ phi + 0.01 * np.eye(p)) @ phi.T @ Y_train
    theta_opt = np.linalg.pinv(phi.T @ phi + lambda_opt * np.eye(p)) @ phi.T @ Y_train
    
    Y_test_pred_fixed = phi_test @ theta_fixed
    Y_test_pred_opt = phi_test @ theta_opt
    
    errors_fixed_lambda.append(((Y_test-Y_test_pred_fixed)**2).mean())
    errors_opt_lambda.append(((Y_test-Y_test_pred_opt)**2).mean())
"""
Step 3 : Plot the results
"""    

plt.figure(figsize = (24, 8))
plt.rc('text', usetex = True)
plt.rc('font', family = 'serif')
plt.rc('font', size = 24)


plt.scatter(num_params, errors_fixed_lambda, color = 'black',
            label = r"Test error with fixed $\lambda = 0.01$",
            ) 
plt.legend()

plt.plot(num_params, errors_opt_lambda, 'k', label = r"Test error with tuned $\lambda$")
plt.legend()
plt.xlabel(r'$\#$ parameters')
plt.ylabel('Test error')
plt.title(r'Test error vs. $\#$ params')

plt.savefig('double_descent.png')
plt.show()

LinAlgError: SVD did not converge